In [1]:
from dataloaders import data_set,data_dict
import torch
import yaml
import os
import torch.nn as nn

import torch
import torch.optim as optim

from dataloaders import data_set,data_dict
from models.model_builder import model_builder
import pandas as pd

import numpy as np

import ray
from ray import tune, air
from ray.air import session, Checkpoint
from ray.tune.examples.mnist_pytorch import ConvNet, get_data_loaders, test
from ray.tune.schedulers import PopulationBasedTraining

import random
from ray.tune.schedulers.pb2 import PB2

from pbtsearch import train

from analysis import cal_test_accuracy

/home/janis/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-18 18:34:46,989	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-18 18:34:47,036	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-18 18:34:47,108	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
config_path = os.path.abspath('configs/data.yaml')
model_config = os.path.abspath('configs/model.yaml')
root_path = os.path.abspath("datasets")
to_save_path = os.path.abspath("ISWC2022LearnableFilter/Run_logs")   
freq_save_path = os.path.abspath("ISWC2022LearnableFilter/Freq_data")
window_save_path = os.path.abspath("ISWC2022LearnableFilter/Sliding_window")

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    def __getattr__(self, attr):
        if attr.startswith('__'):
            raise AttributeError("Should not start with __")
        return self.get(attr)
    #__getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

def get_args():
    args = dotdict()   
    
    
    args.training_iterations = 100
    args.restore = False
    args.perturbation_interval = 3
    args.trials = 16
    args.scheduler = "pbt"
    args.storage_path = "/home/janis/PopulationBasedTraining/RayTuneResults/hapt/pbt"
    args.cpu_per_trial = 1
    args.gpu_per_trial = 0.2
    args.synch = True
    #exploit 
    args.quantile_fraction = 0.25
    args.trainable_api = 'class'
    args.reuse_actor = True
    
    #only pbt
    args.resample_probability = 0.2
    args.burn_in_period = 0
    

    args.seed = 1

    # TODO change the path as relative path
    args.to_save_path     = to_save_path
    args.freq_save_path   = freq_save_path
    args.window_save_path = window_save_path
    args.root_path        = root_path

    args.drop_transition  = False
    args.datanorm_type    = "standardization" # None ,"standardization", "minmax"

    args.batch_size       = 256                                                    
    args.shuffle          = True
    args.drop_last        = False
    args.train_vali_quote = 0.90                                           


    # training setting 
    #args.train_epochs            = 150

    args.learning_rate           = 0.001  
    args.learning_rate_patience  = 5
    args.learning_rate_factor    = 0.1


    args.early_stop_patience     = 15

    #args.use_gpu                 = True if torch.cuda.is_available() else False
    #args.gpu                     = 0
    #args.use_multi_gpu           = False

    args.optimizer               = "Adam"
    args.criterion               = "CrossEntropy"

    args.data_name                        =  "hapt"
    args.use_vali_keys = True
    

    args.wavelet_filtering                = False
    args.wavelet_filtering_regularization = False
    args.wavelet_filtering_finetuning     = False


    args.difference       = False 
    args.filtering        = False
    args.magnitude        = False
    args.weighted_sampler = False

    args.pos_select       = None
    args.sensor_select    = None


    args.representation_type = "time"
    #args.exp_mode            = "LOCV"
    args.exp_mode            = "Given"

  #  if args.data_name      ==  "skodar":
  #      args.exp_mode            = "SOCV"

    config_file = open(config_path, mode='r')
    data_config = yaml.load(config_file, Loader=yaml.FullLoader)
    config = data_config[args.data_name]

    args.root_path       = os.path.join(args.root_path,config["filename"])
    args.sampling_freq   = config["sampling_freq"]
    args.num_classes     =  config["num_classes"]
    window_seconds       = config["window_seconds"]
    args.windowsize      =   int(window_seconds * args.sampling_freq) 
    args.input_length    =  args.windowsize
    # input information
    args.c_in            = config["num_channels"]

    if args.wavelet_filtering :
        
        if args.windowsize%2==1:
            N_ds = int(torch.log2(torch.tensor(args.windowsize-1)).floor()) - 2
        else:
            N_ds = int(torch.log2(torch.tensor(args.windowsize)).floor()) - 2

        args.f_in            =  args.number_wavelet_filtering*N_ds+1
    else:
        args.f_in            =  1


    args.filter_scaling_factor = 0.25
    args.model_type            = "deepconvlstm"
    args.model_config = model_config

    #args.random_augmentation_prob = 0.5
    #args.mixup_probability  = 0.5
    #args.random_aug_first = True
    args.mixup_alpha  = 0.4
    args.mixup_argmax = True

    args.max_aug = 3

    args.random_augmentation_config = {"jitter":True,
                                    "moving_average":True,
                                    "magnitude_scaling":True,
                                    "magnitude_warp":True,
                                    "magnitude_shift":True,
                                    "time_warp":True,
                                    "window_warp":True,
                                    "window_slice":True,
                                    "random_sampling":True,
                                    "slope_adding":True
                                    }
    random_augmentation_nr = 0
    for key in args.random_augmentation_config.keys():
        if args.random_augmentation_config[key]==1:
            random_augmentation_nr = random_augmentation_nr+1
    args.random_augmentation_nr = random_augmentation_nr

    return args

In [3]:
args = get_args()
result_grid = train(args)

(RayModel pid=54656) Train data number :  9149
(RayModel pid=54656) The number of classes is :  12
(RayModel pid=54656) The input_length  is :  128
(RayModel pid=54656) The channel_in is :  6
(RayModel pid=54656) Validation data number :  1319
(RayModel pid=54656) Build the DeepConvLSTM model!
(RayModel pid=54656) new best score!!!!
(RayModel pid=54759) Train data number :  9149 [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RayModel pid=54759) The number of classes is :  12 [repeated 4x across cluster]
(RayModel pid=54759) The input_length  is :  128 [repeated 4x across cluster]
(RayModel pid=54759) The channel_in is :  6 [repeated 4x across cluster]
(RayModel pid=54759) Validation data number :  1319 [repeated 4x across cluster]
(RayModel pid=54759) Build the DeepConvLSTM model! [repeated 4x across cluster]
(

2023-08-18 18:35:58,806	INFO pbt.py:809 -- 

[PopulationBasedTraining] [Exploit] Cloning trial 28f9e_00013 (score = 0.840788) into trial 28f9e_00002 (score = 0.695982)

2023-08-18 18:35:58,806	INFO pbt.py:836 -- 

[PopulationBasedTraining] [Explore] Perturbed the hyperparameter config of trial28f9e_00002:
random_augmentation_prob : 0.9 --- (shift left) --> 0.8
mixup_probability : 0.6000000000000001 --- (shift right) --> 0.7000000000000001

2023-08-18 18:35:58,894	INFO pbt.py:809 -- 

[PopulationBasedTraining] [Exploit] Cloning trial 28f9e_00015 (score = 0.844579) into trial 28f9e_00001 (score = 0.701289)

2023-08-18 18:35:58,894	INFO pbt.py:836 -- 

[PopulationBasedTraining] [Explore] Perturbed the hyperparameter config of trial28f9e_00001:
random_augmentation_prob : 0.6000000000000001 --- (shift right) --> 0.7000000000000001
mixup_probability : 0.4 --- (shift right) --> 0.5

2023-08-18 18:35:58,980	INFO pbt.py:809 -- 

[PopulationBasedTraining] [Exploit] Cloning trial 28f9e_00010 (sco

(RayModel pid=54656) new best score!!!! [repeated 6x across cluster]


2023-08-18 18:35:59,065	INFO pbt.py:809 -- 

[PopulationBasedTraining] [Exploit] Cloning trial 28f9e_00013 (score = 0.840788) into trial 28f9e_00000 (score = 0.755118)

2023-08-18 18:35:59,066	INFO pbt.py:836 -- 

[PopulationBasedTraining] [Explore] Perturbed the hyperparameter config of trial28f9e_00000:
random_augmentation_prob : 0.9 --- (shift right) --> 1.0
mixup_probability : 0.6000000000000001 --- (resample) --> 0.7000000000000001



(RayModel pid=55183) Train data number :  9149
(RayModel pid=55183) The number of classes is :  12
(RayModel pid=55183) The input_length  is :  128
(RayModel pid=55183) The channel_in is :  6
(RayModel pid=55183) Validation data number :  1319
(RayModel pid=55183) Build the DeepConvLSTM model!


...

(RayModel pid=55183) new best score!!!! [repeated 2x across cluster]
(RayModel pid=55228) Train data number :  9149
(RayModel pid=55228) The number of classes is :  12
(RayModel pid=55228) The input_length  is :  128
(RayModel pid=55228) The channel_in is :  6
(RayModel pid=55228) Validation data number :  1319
(RayModel pid=55228) Build the DeepConvLSTM model!
(RayModel pid=55283) Train data number :  9149
(RayModel pid=55283) The number of classes is :  12
(RayModel pid=55283) The input_length  is :  128
(RayModel pid=55283) The channel_in is :  6
(RayModel pid=55283) Validation data number :  1319
(RayModel pid=55283) Build the DeepConvLSTM model!
(RayModel pid=55183) Learning rate adjusting counter: 1 out of 5


(RayModel pid=55317) Restored on 10.12.179.67 from checkpoint: /tmp/checkpoint_tmp_a5831cda4a5942648a4809e1e7919baa [repeated 3x across cluster]
(RayModel pid=55317) Current state after restoring: {'_iteration': 3, '_timesteps_total': None, '_time_total': 19.005426168441772, '_episodes_total': None} [repeated 3x across cluster]


(RayModel pid=55183) Learning rate adjusting counter: 2 out of 5
(RayModel pid=55228) new best score!!!! [repeated 2x across cluster]
(RayModel pid=55376) Train data number :  9149 [repeated 2x across cluster]
(RayModel pid=55376) The number of classes is :  12 [repeated 2x across cluster]
(RayModel pid=55376) The input_length  is :  128 [repeated 2x across cluster]
(RayModel pid=55376) The channel_in is :  6 [repeated 2x across cluster]
(RayModel pid=55317) Validation data number :  1319
(RayModel pid=55317) Build the DeepConvLSTM model!
(RayModel pid=55376) Validation data number :  1319
(RayModel pid=55376) Build the DeepConvLSTM model!
(RayModel pid=55228) Learning rate adjusting counter: 1 out of 5
(RayModel pid=55228) Learning rate adjusting counter: 2 out of 5
(RayModel pid=55317) new best score!!!! [repeated 2x across cluster]
(RayModel pid=55415) Train data number :  9149
(RayModel pid=55415) The number of classes is :  12
(RayModel pid=55415) The input_length  is :  128
(RayM

(RayModel pid=55317) Restored on 10.12.179.67 from checkpoint: /tmp/checkpoint_tmp_ab123fa3343e42d5b24449c2abd2dfe6 [repeated 3x across cluster]
(RayModel pid=55317) Current state after restoring: {'_iteration': 3, '_timesteps_total': None, '_time_total': 18.043455839157104, '_episodes_total': None} [repeated 3x across cluster]


(RayModel pid=55516) Train data number :  9149
(RayModel pid=55516) The number of classes is :  12
(RayModel pid=55516) The input_length  is :  128
(RayModel pid=55516) The channel_in is :  6
(RayModel pid=55516) Validation data number :  1319
(RayModel pid=55516) Build the DeepConvLSTM model!
(RayModel pid=55536) Train data number :  9149
(RayModel pid=55536) The number of classes is :  12
(RayModel pid=55536) The input_length  is :  128
(RayModel pid=55536) The channel_in is :  6
(RayModel pid=55536) Validation data number :  1319
(RayModel pid=55536) Build the DeepConvLSTM model!


In [ ]:
best_result = result_grid.get_best_result(mode="max", metric="mean_accuracy")
best_checkpoint = best_result.checkpoint
checkpoint_dict = best_checkpoint.to_dict()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

total_loss,  acc, f_w,  f_macro, f_micro = cal_test_accuracy(args, device, checkpoint_dict)

print(total_loss)
print(acc)

2023-08-18 18:27:15,989	WARNING experiment_analysis.py:783 -- Could not find best trial. Did you pass the correct `metric` parameter?


RuntimeError: No best trial found for the given metric: mean_accuracy. This means that no trial has reported this metric, or all values reported for this metric are NaN. To not ignore NaN values, you can set the `filter_nan_and_inf` arg to False.